In [4]:
import numpy as np
import pandas as pd
#import library numpy dan pandas

In [5]:
df=pd.read_csv('emositwiter.csv',sep=',')
#read csv file

In [6]:
df.head()
#verifikasi 5 row data pertama saja

,label,tweet
0,anger,"Soal jln Jatibaru,polisi tdk bs GERTAK gubernu..."
1,anger,"Sesama cewe lho (kayaknya), harusnya bisa lebi..."
2,happy,Kepingin gudeg mbarek Bu hj. Amad Foto dari go...
3,anger,"Jln Jatibaru,bagian dari wilayah Tn Abang.Peng..."
4,happy,"Sharing pengalaman aja, kemarin jam 18.00 bata..."


In [7]:
df.tail()
#verifikasi 5 row data terakhir saja

,label,tweet
4396,love,"Tahukah kamu, bahwa saat itu papa memejamkan m..."
4397,fear,Sulitnya menetapkan Calon Wapresnya Jokowi di ...
4398,anger,"5. masa depannya nggak jelas. lha iya, gimana ..."
4399,happy,[USERNAME] dulu beneran ada mahasiswa Teknik U...
4400,sadness,"Ya Allah, hanya Engkau yang mengetahui rasa sa..."


In [8]:
df.columns
#verifikasi nama kolom

Index(['label', 'tweet'], dtype='object')

In [9]:
df.isnull().sum()
#verifikasi kelengkapan data

label    0
tweet    0
dtype: int64

In [10]:
len(df)
#verdifikasi panjang row data

4401

In [11]:
df['label'].value_counts()
#melihat unik value dari kolom label nya

anger      1101
happy      1017
sadness     997
fear        649
love        637
Name: label, dtype: int64

Preprocesing dengan librari sastrawi

In [12]:


from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
def remove_punctuation(text):
    import string
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

df['tweet'] = df['tweet'].apply(remove_punctuation)

factory = StopWordRemoverFactory()
sw = factory.get_stop_words()
def stopwords(text):
    text = [word.lower() for word in text.split() if word.lower() not in sw]
    return " ".join(text)
df['tweet'] = df['tweet'].apply(stopwords)

factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemming(text):    
    text = [stemmer.stem(word) for word in text.split()]
    return " ".join(text)

df['tweet'] = df['tweet'].apply(stemming)

In [14]:
import sklearn

In [15]:
from sklearn.model_selection import train_test_split
#library untuk split data test dan data training

In [16]:
X = df['tweet']

In [17]:
y = df['label']
#label 'keluhan', 'respon' atau 'bukan keluhan atau respon'

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.30, random_state=42)
#dipakai 30% untuk test size, 70% nya training size

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [20]:
from sklearn.naive_bayes import MultinomialNB
#dipakai algoritma naive bayes

In [21]:
from sklearn.pipeline import Pipeline
#Pipeline kelas perform kombinasi vectorizer dan tfidf dalam satu syntax

In [22]:
text_clf = Pipeline([('tfidf',TfidfVectorizer()),('clf',MultinomialNB())])
#semua step dalam satu single step

In [23]:
text_clf.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [24]:
predictions = text_clf.predict(X_test)

In [25]:
from sklearn.metrics import confusion_matrix,classification_report

In [26]:
print(confusion_matrix(y_test,predictions))

[[288   1  10   0  51]
 [ 54  48  15   2  62]
 [ 57   0 186   2  62]
 [ 12   1  36  66  76]
 [ 69   0  24   3 196]]


In [27]:
print(classification_report(y_test,predictions))
#statistik hasil test dari training

              precision    recall  f1-score   support

       anger       0.60      0.82      0.69       350
        fear       0.96      0.27      0.42       181
       happy       0.69      0.61      0.64       307
        love       0.90      0.35      0.50       191
     sadness       0.44      0.67      0.53       292

    accuracy                           0.59      1321
   macro avg       0.72      0.54      0.56      1321
weighted avg       0.68      0.59      0.58      1321



In [28]:
from sklearn import metrics

In [31]:
metrics.accuracy_score(y_test,predictions)
#tingkat akurasi nya

0.5934897804693414

In [32]:
text_clf.predict(["Buka twitter muncul promote ginian. Gimana atuh kalo anak kecil yg liat? Cc: @Menkominfo @ridwankamil https://t.co/3okfeZIsap"])
#masukkan tweet uji di sini - belum pakai GUI

array(['anger'], dtype='<U7')